In [1]:
import numpy as np
import h5py
import awkward as ak
import pickle

In [3]:
pickles = ['rechit_x', 'rechit_y', 'rechit_z', 'rechit_energy', 'all_target']

The following lines show the conversion from pickles to HDF5 format.

In [ ]:
path_to_pickles='/home/rusack/shared/pickles/hgcal_electron/flat_0001/'
path_to_hdf5='/home/rusack/shared/hdf5/hgcal_electron/flat_0001/'

In [ ]:
arrays = {}
for ip, p in enumerate(pickles):
    filename = '{}/{}.pickle'.format(path_to_pickles, p)
    with open(filename, 'rb') as file_:
        arrays[p] = pickle.load(file_)

nhits = ak.count(ak.ones_like(arrays['rechit_x']), axis=1)
arrays['nhits'] = nhits

hf = h5py.File(path_to_hdf5+'hgcal_electron_data.h5', 'w')
for p in pickles+['nhits']:
    if p=='all_target' or p=='nhits': hf.create_dataset('target', data=arrays[p], dtype="float")
    else: hf.create_dataset(p, data=ak.flatten(arrays[p]), dtype="float")
hf.close()

The following lines show the conversion from HDF5 to pickles.

In [10]:
path_to_pickles='/home/rusack/shared/pickles/hgcal_electron/hdf_test/'
path_to_hdf5='/home/rusack/shared/hdf5/hgcal_electron/flat_0001/'

In [4]:
hf = h5py.File(path_to_hdf5+'hgcal_electron_data.h5', 'r')

In [12]:
arrays = {}
arrays['all_target'] = np.asarray(hf['target'])
arrays['nhits'] = np.asarray(hf['nhits'])
for p in pickles:
    if p=='all_target': continue
    A = np.asarray(hf[p])
    arrays[p] = ak.unflatten(A, arrays['nhits'])
    with open(path_to_pickles+p+'.pickle', 'wb') as f0:
        pickle.dump(arrays[p], f0, protocol=4)

In [11]:
with open(path_to_pickles+'all_target.pickle', 'wb') as f0:
    pickle.dump(arrays['all_target'], f0)

In [ ]:
hf.close()